In [1]:
import sys
sys.path.append('..')
from polynomial_program import PolynomialProgram

In [2]:
import functools
from collections import OrderedDict

# IBMQ.load_account()
import numpy as np
from qiskit import Aer
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.aqua.algorithms import VQE, QAOA
from qiskit.aqua.components.optimizers import SPSA
from qiskit.circuit.library import RealAmplitudes

In [3]:
M = [2, 6, 3]
K = [1, 4, 2]
T = [12, 6, 24]
d = 19


def get_time_matrix(M, T):
    r = []
    for i in M:
        tmp = []
        for j in T:
            tmp.append(j / i)
        r.append(tmp)
    return np.array(r)


def get_cost_matrix(time_matrix, K):
    m = []
    for i in range(len(time_matrix)):
        tmp = []
        for j in time_matrix[i]:
            tmp.append(K[i] * j)
        m.append(tmp)
    return m


time_matrix = np.array(get_time_matrix(M, T))
cost_matrix = np.array(get_cost_matrix(time_matrix, K))

print("Time matrix:\n {}".format(time_matrix))
print("Cost matrix:\n {}".format(cost_matrix))

Time matrix:
 [[ 6.  3. 12.]
 [ 2.  1.  4.]
 [ 4.  2.  8.]]
Cost matrix:
 [[ 6.  3. 12.]
 [ 8.  4. 16.]
 [ 8.  4. 16.]]


In [4]:
def sample_most_likely(state_vector):
    if isinstance(state_vector, (OrderedDict, dict)):
        # get the binary string with the largest count
        binary_string = sorted(state_vector.items(), key=lambda kv: kv[1])
        repetitions = int(binary_string[-1][1])
        binary_string = binary_string[-1][0]
        x = np.asarray([int(y) for y in reversed(list(binary_string))])
        return x, repetitions
    return [], 0


optimal_key = "0000001000"


def get_stats_for_result(dict_res):
    optimal = 0
    correct = 0
    incorrect = 0
    correct_config = 0
    incorrect_config = 0

    if optimal_key in dict_res:
        optimal = dict_res[optimal_key]
    for key, val in dict_res.items():
        key = key[::-1]
        if is_correct(key):
            correct += val
            correct_config += 1
        else:
            incorrect += val
            incorrect_config += 1

    print('most likely solution: ', sample_most_likely(dict_res))
    print("optimal: ", optimal)
    print("correct solutions: ", correct)
    print("incorrect solutions: ", incorrect)
    print("correct configs: ", correct_config)
    print("incorrect configs: ", incorrect_config)


def is_correct(key):
    return solution_vector_correct(key) and execution_time(key) == d


correct_machines = ['00', '01', '10']
machine_to_index = {'00': 0, '01': 1, '10': 2}


def solution_vector_correct(vector):
    task1_machine = vector[0:2]
    task2_machine = vector[2:4]
    task3_machine = vector[4:6]

    return task1_machine in correct_machines \
        and task2_machine in correct_machines \
        and task3_machine in correct_machines


def execution_time(k):
    task1_machine = machine_to_index.get(k[0:2])
    task2_machine = machine_to_index.get(k[2:4])
    task3_machine = machine_to_index.get(k[4:6])

    
    task1_time = time_matrix[task1_machine, 0] if task1_machine is not None else 0
    task2_time = time_matrix[task2_machine, 1] if task2_machine is not None else 0
    task3_time = time_matrix[task3_machine, 2] if task3_machine is not None else 0
    

    slack_sum = int(k[6]) * 8 + int(k[7]) * 4 + int(k[8]) * 2 + int(k[9]) * 1

    return task1_time + task2_time + task3_time + slack_sum


def execution_cost(k):
    task1_machine = machine_to_index.get(k[0:2])
    task2_machine = machine_to_index.get(k[2:4])
    task3_machine = machine_to_index.get(k[4:6])

    
    task1_cost = cost_matrix[task1_machine, 0] if task1_machine is not None else 0
    task2_cost = cost_matrix[task2_machine, 1] if task2_machine is not None else 0
    task3_cost = cost_matrix[task3_machine, 2] if task3_machine is not None else 0
    

    return task1_cost + task2_cost + task3_cost


def incorrect_machine_count(k):
    task1_machine = machine_to_index.get(k[0:2])
    task2_machine = machine_to_index.get(k[2:4])
    task3_machine = machine_to_index.get(k[4:6])


    return (0 if k[0:2] in correct_machines else 1) \
         + (0 if k[2:4] in correct_machines else 1) \
         + (0 if k[4:6] in correct_machines else 1)

In [5]:
def get_cost_model(x):
    return sum([
            cost_matrix[0, i] * (1 - x[2 * i]) * (1 - x[2 * i + 1])
            + cost_matrix[1, i] * (1 - x[2 * i]) * x[2 * i + 1]
            + cost_matrix[2, i] * x[2 * i] * (1 - x[2 * i + 1])
            for i in range(0, 3)
    ])


def get_machine_usage_model(x):
    return sum([
        x[2 * i] * x[2 * i + 1] for i in range(0, 3)
    ])


def get_deadline_model(x):
    time_sum = sum([
                  time_matrix[0, i] * (1 - x[2 * i]) * (1 - x[2 * i + 1])
                + time_matrix[1, i] * (1 - x[2 * i]) * x[2 * i + 1]
                + time_matrix[2, i] * x[2 * i] * (1 - x[2 * i + 1])
                for i in range(0, 3)
    ])
    slack_sum = 8 * x[6] + 4 * x[7] + 2 * x[8] + x[9]
    time_constraint = (d - time_sum - slack_sum) ** 2
    
    return time_constraint

In [6]:
def compute_eigenvalues(quibit_op):
    from qiskit.aqua.algorithms import NumPyEigensolver
    count = 1024
    eigensolver = NumPyEigensolver(qubit_op, count)
    eigensolver_result = eigensolver.compute_eigenvalues()
    print('state\t\ttime\tcost\tmachine use\tcorrect\teigenvalue')
    for eigenstate, eigenvalue in zip(eigensolver_result.eigenstates, eigensolver_result.eigenvalues):
        eigenstate, = eigenstate.sample().keys()
        eigenstate = eigenstate[::-1]
        eigenvalue = eigenvalue
        print(f'{eigenstate}\t{execution_time(eigenstate)}\t{execution_cost(eigenstate)}', end='')
        print(f'\t{incorrect_machine_count(eigenstate)}\t\t{is_correct(eigenstate)}\t{eigenvalue}')

In [7]:
A = 1
B = 20
C = 10

In [8]:
pp = PolynomialProgram(10)

pp.add_objective(get_cost_model(pp.x), A)
pp.add_objective(get_machine_usage_model(pp.x), B)
pp.add_objective(get_deadline_model(pp.x), C)

In [9]:
qubit_op, offset = pp.to_ising()

In [10]:
compute_eigenvalues(qubit_op)

state		time	cost	machine use	correct	eigenvalue
0001000000	19.0	22.0	0		True	(-22.24999999999978+0j)
1000000000	19.0	23.0	0		True	(-21.249999999999822+0j)
0100000010	19.0	23.0	0		True	(-21.24999999999968+0j)
1001000010	19.0	24.0	0		True	(-20.249999999999837+0j)
1010000001	19.0	24.0	0		True	(-20.24999999999983+0j)
0101000100	19.0	24.0	0		True	(-20.249999999999822+0j)
0110000011	19.0	24.0	0		True	(-20.249999999999815+0j)
0000100010	19.0	25.0	0		True	(-19.250000000000043+0j)
0000010110	19.0	25.0	0		True	(-19.2499999999999+0j)
0001100100	19.0	26.0	0		True	(-18.25000000000003+0j)
0010100011	19.0	26.0	0		True	(-18.25000000000002+0j)
0001011000	19.0	26.0	0		True	(-18.249999999999915+0j)
0010010111	19.0	26.0	0		True	(-18.249999999999908+0j)
0100011010	19.0	27.0	0		True	(-17.250000000000043+0j)
1000100100	19.0	27.0	0		True	(-17.24999999999997+0j)
1000011000	19.0	27.0	0		True	(-17.2499999999999+0j)
0100100110	19.0	27.0	0		True	(-17.249999999999886+0j)
0101011100	19.0	28.0	0		True	(-16.2500000000

0001000010	21.0	22.0	0		False	(17.750000000000234+0j)
1111001000	20.0	12.0	2		False	(17.75000000000025+0j)
1000000010	21.0	23.0	0		False	(18.750000000000178+0j)
0100000000	17.0	23.0	0		False	(18.750000000000206+0j)
0100000100	21.0	23.0	0		False	(18.75000000000022+0j)
1001000000	17.0	24.0	0		False	(19.750000000000163+0j)
1001000100	21.0	24.0	0		False	(19.750000000000163+0j)
1010000011	21.0	24.0	0		False	(19.75000000000017+0j)
0101000010	17.0	24.0	0		False	(19.750000000000178+0j)
0110000001	17.0	24.0	0		False	(19.750000000000185+0j)
0110000101	21.0	24.0	0		False	(19.750000000000185+0j)
0101000110	21.0	24.0	0		False	(19.75000000000029+0j)
0000100000	17.0	25.0	0		False	(20.749999999999957+0j)
0000100100	21.0	25.0	0		False	(20.749999999999957+0j)
0000011000	21.0	25.0	0		False	(20.750000000000085+0j)
0000010100	17.0	25.0	0		False	(20.7500000000001+0j)
0010100101	21.0	26.0	0		False	(21.749999999999964+0j)
0001100110	21.0	26.0	0		False	(21.74999999999997+0j)
0010100001	17.0	26.0	0		False	(21.7

1010101001	23.0	28.0	0		False	(143.7499999999999+0j)
1001011110	23.0	28.0	0		False	(143.74999999999997+0j)
0110011111	23.0	28.0	0		False	(143.74999999999997+0j)
1001101010	23.0	28.0	0		False	(143.75+0j)
1001100010	15.0	28.0	0		False	(143.75000000000003+0j)
1010100001	15.0	28.0	0		False	(143.75000000000003+0j)
0110100011	15.0	28.0	0		False	(143.75000000000003+0j)
0110010111	15.0	28.0	0		False	(143.75000000000003+0j)
0101101100	23.0	28.0	0		False	(143.75000000000009+0j)
1010011101	23.0	28.0	0		False	(143.75000000000009+0j)
0110101011	23.0	28.0	0		False	(143.75000000000009+0j)
1010010101	15.0	28.0	0		False	(143.75000000000014+0j)
0101100100	15.0	28.0	0		False	(143.75000000000014+0j)
1001010110	15.0	28.0	0		False	(143.75000000000014+0j)
0000111110	23.0	9.0	1		False	(144.7499999999998+0j)
0000110110	15.0	9.0	1		False	(144.7499999999998+0j)
0010110111	15.0	10.0	1		False	(145.7499999999998+0j)
0001111000	15.0	10.0	1		False	(145.7499999999998+0j)
0010111111	23.0	10.0	1		False	(145.749999999999

1110111011	13.0	4.0	2		False	(359.7499999999998+0j)
1101111100	13.0	4.0	2		False	(359.7499999999998+0j)
1011100001	13.0	24.0	1		False	(359.75+0j)
0111010111	13.0	24.0	1		False	(359.75+0j)
0111101111	25.0	24.0	1		False	(359.75+0j)
1011101101	25.0	24.0	1		False	(359.75+0j)
0111100011	13.0	24.0	1		False	(359.75+0j)
1011010101	13.0	24.0	1		False	(359.7500000000001+0j)
0011110111	13.0	6.0	2		False	(361.7499999999998+0j)
1011111001	13.0	8.0	2		False	(363.7499999999998+0j)
0111111011	13.0	8.0	2		False	(363.7499999999998+0j)
1111001101	25.0	12.0	2		False	(367.7500000000002+0j)
1111000001	13.0	12.0	2		False	(367.7500000000002+0j)
1111011001	13.0	16.0	2		False	(371.75+0j)
1111100101	13.0	16.0	2		False	(371.7500000000001+0j)
1111111101	13	0	3		False	(375.7499999999998+0j)
0000000101	26.0	21.0	0		False	(466.7500000000002+0j)
0001000111	26.0	22.0	0		False	(467.7500000000002+0j)
0010000110	26.0	22.0	0		False	(467.7500000000002+0j)
1000000111	26.0	23.0	0		False	(468.7500000000001+0j)
0100001001	26.0	

0100110101	10.0	11.0	1		False	(796.7499999999997+0j)
1000110011	10.0	11.0	1		False	(796.7499999999998+0j)
1001110101	10.0	12.0	1		False	(797.7499999999998+0j)
0101110111	10.0	12.0	1		False	(797.7499999999998+0j)
1010110100	10.0	12.0	1		False	(797.7499999999998+0j)
0110110110	10.0	12.0	1		False	(797.7499999999998+0j)
1100001101	28.0	15.0	1		False	(800.7500000000002+0j)
1101001111	28.0	16.0	1		False	(801.7500000000001+0j)
1110001110	28.0	16.0	1		False	(801.7500000000002+0j)
0011001010	28.0	18.0	1		False	(803.7500000000002+0j)
1100010011	10.0	19.0	1		False	(804.75+0j)
1110100000	10.0	20.0	1		False	(805.75+0j)
1110010100	10.0	20.0	1		False	(805.75+0j)
1101100001	10.0	20.0	1		False	(805.75+0j)
1101010101	10.0	20.0	1		False	(805.75+0j)
1011001100	28.0	20.0	1		False	(805.7500000000001+0j)
0111001110	28.0	20.0	1		False	(805.7500000000002+0j)
0011010000	10.0	22.0	1		False	(807.75+0j)
0011101110	28.0	22.0	1		False	(807.75+0j)
1100110111	10.0	3.0	2		False	(808.7499999999998+0j)
1101111001	10.0	4.

In [11]:
seed = 10598
aqua_globals.random_seed = seed

max_trials = 1000
shots = 1000
reps = 2

entanglement = 'full'

In [12]:
spsa = SPSA(maxiter=max_trials)
ry = RealAmplitudes(qubit_op.num_qubits, reps=reps, entanglement=entanglement)
vqe = VQE(qubit_op, ry, spsa)

# provider = IBMQ.get_provider('ibm-q')
backend = Aer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, seed_simulator=seed, seed_transpiler=seed, shots=shots)
result_vqe = vqe.run(quantum_instance)   
state_vector_vqe = result_vqe['eigenstate']
get_stats_for_result(state_vector_vqe)

most likely solution:  (array([1, 0, 0, 1, 0, 0, 0, 0, 0, 1]), 88)
optimal:  2
correct solutions:  102
incorrect solutions:  898
correct configs:  7
incorrect configs:  83


In [ ]:
spsa = SPSA(maxiter=max_trials)

qaoa = QAOA(qubit_op, optimizer=spsa, p=1)

# provider = IBMQ.get_provider('ibm-q')
backend = Aer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, seed_simulator=seed, seed_transpiler=seed, shots=shots)
result = qaoa.run(quantum_instance)   
state_vector = result['eigenstate']
get_stats_for_result(state_vector)